<a href="https://colab.research.google.com/github/victorsa2/sales_meli/blob/main/sales_meli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício 1

## Preparando o Ambiente no Github

In [ ]:
!git clone https://github.com/victorsa2/sales_meli.git

Cloning into 'sales_meli'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 80 (delta 33), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 467.51 KiB | 5.57 MiB/s, done.
Resolving deltas: 100% (33/33), done.


## Subindo o arquivo .env com as variáveis no drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


##Instalando Dependencias não disponíveis no colab

In [ ]:
pip install python-dotenv

In [ ]:
import requests
import webbrowser
from dotenv import load_dotenv
import os

##Consultando API do Mercado Livre

##Obter Access Token e User ID

In [ ]:
load_dotenv('/content/.env')

APP_ID = os.getenv("APP_ID")
APP_SECRET = os.getenv("APP_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")

def get_access_token_and_user_id():

    auth_url = f'https://auth.mercadolivre.com.br/authorization?response_type=code&client_id={APP_ID}&redirect_uri={REDIRECT_URI}'
    return auth_url

def request_token(authorization_code):
    token_url = 'https://api.mercadolibre.com/oauth/token'
    payload = {
        'grant_type': 'authorization_code',
        'client_id': os.getenv("APP_ID"),
        'client_secret': os.getenv("APP_SECRET"),
        'code': authorization_code,
        'redirect_uri': os.getenv("REDIRECT_URI")
    }

    response = requests.post(token_url, data=payload)
    token_data = response.json()

    access_token = token_data['access_token']
    user_id = token_data['user_id']

    return access_token, user_id

if __name__ == "__main__":
    auth_url = get_access_token_and_user_id()
    print("Abra o seguinte URL no seu navegador para obter o código de autorização:")
    print(auth_url)

    authorization_code = input("Insira o código de autorização: ")
    access_token, user_id = request_token(authorization_code)


Abra o seguinte URL no seu navegador para obter o código de autorização:
https://auth.mercadolivre.com.br/authorization?response_type=code&client_id=5500572046072825&redirect_uri=https://victorsa.netlify.app
Insira o código de autorização: TG-66fc77ceadd72400012c00ad-478295390


##Obter dimensão de Categorias

In [ ]:
import requests
import pandas as pd

def fetch_categories():
    url = "https://api.mercadolibre.com/sites/MLB/categories"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao obter categorias: {response.status_code}")
        return []

categories_data = fetch_categories()
categories_list = []

for category in categories_data:
    categories_list.append({
        'id_category': category['id'],
        'name': category['name'],
    })

dim_Category = pd.DataFrame(categories_list)
dim_Category.to_csv('category.csv',encoding='utf-8-sig')

print("Dimensão Categorias:")
print(dim_Category.head())

Dimensão Categorias:
  id_category                      name
0     MLB5672  Acessórios para Veículos
1   MLB271599                      Agro
2     MLB1403       Alimentos e Bebidas
3     MLB1071                   Animais
4     MLB1367   Antiguidades e Coleções


## Consumindo API por produtos para conseguir alguns vendedores de exemplo, pesquisando por todas as categorias do Mercado Livre.


In [ ]:
def fetch_categories():
    url = "https://api.mercadolibre.com/sites/MLB/categories"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter categorias: {e}")
        return []

def fetch_sales_in_category(category_id, access_token, site_id):
    url = f'https://api.mercadolibre.com/sites/{site_id}/search?category={category_id}'
    headers = {'Authorization': f'Bearer {access_token}'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()['results']
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter vendas na categoria {category_id}: {e}")
        return []

ACCESS_TOKEN = access_token
SITE_ID = 'MLB'

sales_list = []
categories_data = fetch_categories()

for category in categories_data:
    category_id = category['id']
    print(f"\nBuscando vendas na categoria: {category['name']} (ID: {category_id})")

    sales_data = fetch_sales_in_category(category_id, ACCESS_TOKEN, SITE_ID)

    if sales_data:
        for item in sales_data:
            if 'seller' in item and 'nickname' in item['seller']:
                sales_list.append({
                    'seller_id': item['seller']['id'],
                    'seller_nickname': item['seller']['nickname'],
                    'category_id': category['id'],
                    'category': category['name'],
                    'item_id': item['id'],
                    'item_name': item['title'],
                    'currency': item['currency_id'],
                    'price': item['price']
                })
    else:
        print("Nenhum resultado encontrado para esta categoria.")

df_sales = pd.DataFrame(sales_list)
df_sales.to_csv('products.csv',encoding='utf-8-sig')

print(df_sales)


Buscando vendas na categoria: Acessórios para Veículos (ID: MLB5672)

Buscando vendas na categoria: Agro (ID: MLB271599)

Buscando vendas na categoria: Alimentos e Bebidas (ID: MLB1403)

Buscando vendas na categoria: Animais (ID: MLB1071)

Buscando vendas na categoria: Antiguidades e Coleções (ID: MLB1367)

Buscando vendas na categoria: Arte, Papelaria e Armarinho (ID: MLB1368)

Buscando vendas na categoria: Bebês (ID: MLB1384)

Buscando vendas na categoria: Beleza e Cuidado Pessoal (ID: MLB1246)

Buscando vendas na categoria: Brinquedos e Hobbies (ID: MLB1132)

Buscando vendas na categoria: Calçados, Roupas e Bolsas (ID: MLB1430)

Buscando vendas na categoria: Câmeras e Acessórios (ID: MLB1039)

Buscando vendas na categoria: Carros, Motos e Outros (ID: MLB1743)

Buscando vendas na categoria: Casa, Móveis e Decoração (ID: MLB1574)

Buscando vendas na categoria: Celulares e Telefones (ID: MLB1051)

Buscando vendas na categoria: Construção (ID: MLB1500)

Buscando vendas na categoria: El

## Exploração e Tratamento

Adicionando o arquivo gerado pelo GPT no ambiente do colab

In [ ]:
import pandas as pd

path_category = '/content/category.csv'
path_products = '/content/products.csv'

df_category = pd.read_csv(path_category, sep=',')
df_products = pd.read_csv(path_products, sep=',')

print('Informações dataframe Category:\n')
shape_category = df_category.info()

print('\nInformações dataframe Products:\n')
shape_products = df_products.info()

Informações dataframe Category:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   32 non-null     int64 
 1   id_category  32 non-null     object
 2   name         32 non-null     object
dtypes: int64(1), object(2)
memory usage: 896.0+ bytes

Informações dataframe Products:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1489 non-null   int64  
 1   seller_id        1489 non-null   int64  
 2   seller_nickname  1489 non-null   object 
 3   category_id      1489 non-null   object 
 4   category         1489 non-null   object 
 5   item_id          1489 non-null   object 
 6   item_name        1489 non-null   object 
 7   currency         1456 non-null   object 
 8   

In [ ]:
category_null = df_category.isnull().sum() > 0
print('Verificando nulos no dataframe Category:\n')
print(category_null)

products_null = df_products.isnull().sum() > 0
print('Verificando nulos no dataframe Products:\n')
print(products_null)

Verificando nulos no dataframe Category:

Unnamed: 0     False
id_category    False
name           False
dtype: bool
Verificando nulos no dataframe Products:

Unnamed: 0         False
seller_id          False
seller_nickname    False
category_id        False
category           False
item_id            False
item_name          False
currency            True
price               True
dtype: bool


In [ ]:
df_price_null = df_products[df_products['price'].isnull()]
print('Linhas com valores nulos na coluna price:\n')
print(df_price_null)

Linhas com valores nulos na coluna price:

      Unnamed: 0  seller_id            seller_nickname category_id  category  \
1396        1396  169213892            BRILHYSEMIJOIAS     MLB1540  Serviços   
1397        1397  209363479                 SUN HAPPY.     MLB1540  Serviços   
1398        1398   29094661                ALUGUEFLUKE     MLB1540  Serviços   
1399        1399   89551550            ZERICARDOGUEDES     MLB1540  Serviços   
1400        1400   42451372             RICARDOYOUSSEF     MLB1540  Serviços   
1401        1401  104637759                DARLAN 1434     MLB1540  Serviços   
1402        1402     519584                  GARIDA_ML     MLB1540  Serviços   
1403        1403   56338640              SAMPA_MVENDAS     MLB1540  Serviços   
1404        1404  177129516             FRETEPONTUALSP     MLB1540  Serviços   
1406        1406  443589211           COMAHOCOMMATHOSP     MLB1540  Serviços   
1407        1407   56338640              SAMPA_MVENDAS     MLB1540  Serviços 


Retirando de todas as tabelas a categoria de "Serviços"

In [ ]:
df_products = df_products[df_products['category'] != 'Serviços']
df_category = df_category[df_category['name'] != 'Serviços']

products_null = df_products.isnull().sum() > 0
print('Verificando nulos no dataframe Products:\n')
print(products_null)

Verificando nulos no dataframe Products:

Unnamed: 0         False
seller_id          False
seller_nickname    False
category_id        False
category           False
item_id            False
item_name          False
currency           False
price              False
dtype: bool


In [ ]:
df_category = df_category.drop(['Unnamed: 0'], axis=1)
df_products = df_products.drop(['Unnamed: 0'], axis=1)

print(df_category)
print(df_products)

   id_category                         name
0      MLB5672     Acessórios para Veículos
1    MLB271599                         Agro
2      MLB1403          Alimentos e Bebidas
3      MLB1071                      Animais
4      MLB1367      Antiguidades e Coleções
5      MLB1368  Arte, Papelaria e Armarinho
6      MLB1384                        Bebês
7      MLB1246     Beleza e Cuidado Pessoal
8      MLB1132         Brinquedos e Hobbies
9      MLB1430    Calçados, Roupas e Bolsas
10     MLB1039         Câmeras e Acessórios
11     MLB1743       Carros, Motos e Outros
12     MLB1574     Casa, Móveis e Decoração
13     MLB1051        Celulares e Telefones
14     MLB1500                   Construção
15     MLB5726             Eletrodomésticos
16     MLB1000   Eletrônicos, Áudio e Vídeo
17     MLB1276           Esportes e Fitness
18   MLB263532                  Ferramentas
19    MLB12404       Festas e Lembrancinhas
20     MLB1144                        Games
21     MLB1459                  

In [ ]:
df_products.to_csv('data_processed/products.csv',encoding='utf-8-sig')
df_category.to_csv('data_processed/category.csv',encoding='utf-8-sig')

##Criação do Banco de Dados SQLite

In [128]:
import sqlite3
import pandas as pd

conn_data = sqlite3.connect('db1.db')
cursor_data = conn_data.cursor()

In [ ]:
# Criação da tabela Categoria

from prettytable import PrettyTable

table = PrettyTable(["id_category", "name"])

cursor_data.execute('''
    CREATE TABLE IF NOT EXISTS raw_category (
        id_category TEXT PRIMARY KEY,
        name TEXT NOT NULL
    )
''')

df_category.to_sql('raw_category', conn_data, if_exists='replace', index=False)

cursor_data.execute('SELECT * FROM raw_category')
rows = cursor_data.fetchall()

for row in rows:
    table.add_row(row)

print(table)


+-------------+-----------------------------+
| id_category |             name            |
+-------------+-----------------------------+
|   MLB5672   |   Acessórios para Veículos  |
|  MLB271599  |             Agro            |
|   MLB1403   |     Alimentos e Bebidas     |
|   MLB1071   |           Animais           |
|   MLB1367   |   Antiguidades e Coleções   |
|   MLB1368   | Arte, Papelaria e Armarinho |
|   MLB1384   |            Bebês            |
|   MLB1246   |   Beleza e Cuidado Pessoal  |
|   MLB1132   |     Brinquedos e Hobbies    |
|   MLB1430   |  Calçados, Roupas e Bolsas  |
|   MLB1039   |     Câmeras e Acessórios    |
|   MLB1743   |    Carros, Motos e Outros   |
|   MLB1574   |   Casa, Móveis e Decoração  |
|   MLB1051   |    Celulares e Telefones    |
|   MLB1500   |          Construção         |
|   MLB5726   |       Eletrodomésticos      |
|   MLB1000   |  Eletrônicos, Áudio e Vídeo |
|   MLB1276   |      Esportes e Fitness     |
|  MLB263532  |         Ferramenta

In [ ]:
#Criando Tabela Produtos

table = PrettyTable(["seller_id", "seller_nickname", "category_id", "category", "item_id", "item_name", "currency", "price"])

cursor_data.execute('''
  CREATE TABLE IF NOT EXISTS raw_product (
      seller_id INTEGER PRIMARY KEY,
      seller_nickname TEXT NOT NULL,
      category_id TEXT NOT NULL,
      category TEXT NOT NULL,
      item_id TEXT NOT NULL,
      item_name TEXT NOT NULL,
      currency TEXT NOT NULL,
      price REAL NOT NULL
    )
''')

df_products.to_sql('raw_product', conn_data, if_exists='replace', index=False)

cursor_data.execute('SELECT * FROM raw_product \
                    LIMIT 5')
rows = cursor_data.fetchall()

for row in rows:
    table.add_row(row)

print(table)

+------------+-----------------+-------------+--------------------------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+
| seller_id  | seller_nickname | category_id |         category         |    item_id    |                                                                          item_name                                                                          | currency | price  |
+------------+-----------------+-------------+--------------------------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+
| 224160443  |  MAF UTILIDADES |   MLB5672   | Acessórios para Veículos | MLB4296308798 |                                               Kit 10 Panos Microfibra Flanela 35*50cm Power Clean Cor Sort

In [ ]:
cursor_data.execute('DROP TABLE IF EXISTS raw_vendas')

In [ ]:
# Criar tabela de vendas

table = PrettyTable(["seller_id", "data_created", "item_id", "quantity", "price"])

cursor_data.execute('''
CREATE TABLE IF NOT EXISTS raw_vendas (
    seller_id INTEGER PRIMARY KEY,
    data_created INTEGER NOT NULL,
    item_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    price REAL NOT NULL
);
''')

vendas = pd.read_csv('/content/data_processed/sales_gpt.csv', sep=';')
vendas.to_sql('raw_vendas', conn_data, if_exists='replace', index=False)

cursor_data.execute('SELECT * FROM raw_vendas \
                    LIMIT 5')
rows = cursor_data.fetchall()

for row in rows:
    table.add_row(row)

print(table)

+------------+----------------------------+---------------+----------+--------+
| seller_id  |        data_created        |    item_id    | quantity | price  |
+------------+----------------------------+---------------+----------+--------+
| 224160443  | 2024-06-22 00:31:45.205367 | MLB4296308798 |    5     |  41.0  |
| 444816473  | 2023-10-20 00:31:45.205433 | MLB3698110641 |    6     |  82.5  |
| 1165626720 | 2024-01-06 00:31:45.205440 | MLB3494040092 |    7     | 184.23 |
| 1148569370 | 2024-06-18 00:31:45.205448 | MLB3721274157 |    8     | 575.0  |
| 183982918  | 2024-07-23 00:31:45.205456 | MLB1381132707 |    8     |  56.0  |
+------------+----------------------------+---------------+----------+--------+


In [ ]:

cursor_data.execute('''
    CREATE VIEW IF NOT EXISTS seller_dim AS
    SELECT
        ROW_NUMBER() OVER(ORDER BY seller_id) AS id_seller,
        seller_id AS cod_seller,
        seller_nickname AS seller_name
    FROM (
        SELECT DISTINCT seller_id, seller_nickname
        FROM raw_product
    ) AS unique_sellers
''')

cursor_data.execute('SELECT * FROM seller_dim \
                      LIMIT 5')

seller_dim = cursor_data.fetchall()
seller_table = PrettyTable()
seller_table.field_names = ["ID Seller", "Cod Seller", "Seller Name"]

for row in seller_dim:
    seller_table.add_row(row)

print("Seller Dimension:")
print(seller_table)

Seller Dimension:
+-----------+------------+--------------------+
| ID Seller | Cod Seller |    Seller Name     |
+-----------+------------+--------------------+
|     1     |   415018   |     CAPITANIO      |
|     2     |   970598   | FUTURO_IMPORTADOS  |
|     3     |  1520446   |     INDAAPARAS     |
|     4     |  1650894   | LIVRARIA-BOA-NOITE |
|     5     |  3920462   | SEBO_PE_NA_ESTRADA |
+-----------+------------+--------------------+


In [ ]:
cursor_data.execute('''
    CREATE VIEW IF NOT EXISTS item_dim AS
    SELECT
        ROW_NUMBER() OVER(ORDER BY item_id) AS id_item,
        item_id AS cod_item,
        item_name,
        category_id AS cod_category,
        category AS category_name
    FROM (
        SELECT DISTINCT item_id, item_name, category_id, category
        FROM raw_product
    ) AS unique_items
''')

cursor_data.execute('SELECT * FROM item_dim \
                    LIMIT 5')
item_dim = cursor_data.fetchall()
item_table = PrettyTable()
item_table.field_names = ["ID Item", "Item ID", "Item Name", "Category_id", "name"]

for row in item_dim:
    item_table.add_row(row)

print("\nItem Dimension:")
print(item_table)


Item Dimension:
+---------+---------------+--------------------------------------------------------------+-------------+-----------+
| ID Item |    Item ID    |                          Item Name                           | Category_id |    name   |
+---------+---------------+--------------------------------------------------------------+-------------+-----------+
|    1    | MLB1028968851 |    2 Caixa Térmica D Isopor Capacidade 120 Litros C/dreno    |  MLB218519  | Ingressos |
|    2    | MLB1030948809 | Caixa Térmica D Isopor Capacidade 165 Litros C/dreno Isoterm |  MLB218519  | Ingressos |
|    3    | MLB1031202677 |      Copo 10ml Acrilico C/1000 Doce Ou Ceia, Degustação      |  MLB218519  | Ingressos |
|    4    | MLB1033727890 | Taça De Champagne Em Acrilico Descartavel 150ml 150 Unidades |  MLB218519  | Ingressos |
|    5    | MLB1044540635 |   Lona De Proteção Transparente 4x50 Metros Media - 1 Rolo   |  MLB218519  | Ingressos |
+---------+---------------+--------------------

In [ ]:
cursor_data.execute('''
    CREATE VIEW IF NOT EXISTS category_dim AS
    SELECT
        ROW_NUMBER() OVER(ORDER BY id_category) AS id_category,
        id_category AS cod_category,
        name AS category_name
    FROM (
        SELECT DISTINCT id_category, name
        FROM raw_category
    ) AS unique_sellers
''')

cursor_data.execute('SELECT * FROM category_dim')

category_dim = cursor_data.fetchall()
category_table = PrettyTable()
category_table.field_names = ["id_category", "cod_category", "category_name"]

for row in category_dim:
    category_table.add_row(row)

print("\nItem Dimension:")
print(category_table)


Item Dimension:
+-------------+--------------+-----------------------------+
| id_category | cod_category |        category_name        |
+-------------+--------------+-----------------------------+
|      1      |   MLB1000    |  Eletrônicos, Áudio e Vídeo |
|      2      |   MLB1039    |     Câmeras e Acessórios    |
|      3      |   MLB1051    |    Celulares e Telefones    |
|      4      |   MLB1071    |           Animais           |
|      5      |   MLB1132    |     Brinquedos e Hobbies    |
|      6      |   MLB1144    |            Games            |
|      7      |   MLB1168    |  Música, Filmes e Seriados  |
|      8      |   MLB1182    |    Instrumentos Musicais    |
|      9      |   MLB1196    |  Livros, Revistas e Comics  |
|      10     |   MLB12404   |    Festas e Lembrancinhas   |
|      11     |   MLB1246    |   Beleza e Cuidado Pessoal  |
|      12     |   MLB1276    |      Esportes e Fitness     |
|      13     |   MLB1367    |   Antiguidades e Coleções   |
|      

In [ ]:
cursor_data.execute('DROP VIEW IF EXISTS FACT_SALES')


In [ ]:
cursor_data.execute('''
    CREATE VIEW FACT_SALES AS
        SELECT
          seller_dim.id_seller AS KEY1_NR_SELLER,
          item_dim.id_item AS KEY1_NR_ITEM,
          raw_vendas.data_created,
          raw_vendas.quantity,
          raw_vendas.price,
          raw_vendas.quantity * raw_vendas.price AS total_sold
        FROM
        raw_vendas
        LEFT OUTER JOIN seller_dim ON raw_vendas.seller_id = seller_dim.cod_seller
        LEFT OUTER JOIN item_dim ON raw_vendas.item_id = item_dim.cod_item

''')

cursor_data.execute('SELECT * FROM FACT_SALES \
                  LIMIT 5')
fact_sales = cursor_data.fetchall()

# Criar a tabela PrettyTable para exibir os resultados
fact_sales_table = PrettyTable()
fact_sales_table.field_names = ["KEY1_NR_SELLER", "KEY1_NR_ITEM", "Data Created", "Quantity", "Price", "Total Sold"]

for row in fact_sales:
    fact_sales_table.add_row(row)

print("\nFact Sales:")
print(fact_sales_table)


Fact Sales:
+----------------+--------------+----------------------------+----------+--------+------------+
| KEY1_NR_SELLER | KEY1_NR_ITEM |        Data Created        | Quantity | Price  | Total Sold |
+----------------+--------------+----------------------------+----------+--------+------------+
|      280       |     984      | 2024-06-22 00:31:45.205367 |    5     |  41.0  |   205.0    |
|      454       |     552      | 2023-10-20 00:31:45.205433 |    6     |  82.5  |   495.0    |
|      783       |     336      | 2024-01-06 00:31:45.205440 |    7     | 184.23 |  1289.61   |
|      775       |     578      | 2024-06-18 00:31:45.205448 |    8     | 575.0  |   4600.0   |
|      218       |      37      | 2024-07-23 00:31:45.205456 |    8     |  56.0  |   448.0    |
+----------------+--------------+----------------------------+----------+--------+------------+


In [ ]:
conn_data.commit()

#Finalização Exercício 1

## Pergunta:

1.   Como você nomearia cada tabela e porque?
      - Para a organização dos dados, inicialmente, criei tabelas com o prefixo "raw", indicando que contêm dados brutos extraídos diretamente da API.

      Em seguida, projetei um Data Warehouse para ser consumido no Tableau, onde desenvolvi views para as Dimensões e a Tabela Fato. O plano inicial consistia em criar dois bancos: um para os dados crus, contendo as dimensões em formato de view, e outro para armazenar as tabelas com os dados já processados. Essa estrutura permitiria realizar consultas de maneira mais eficiente, evitando joins entre views, que podem impactar a performance.

      Contudo, devido ao pequeno volume de dados, decidi implementar tudo em um único banco de dados. As tabelas foram nomeadas para refletir suas funções e o tipo de dados que armazenam, facilitando a compreensão e manutenção do esquema.




# Exercício 2

## 1. No Colab Notebook, escreva consultas SQL para calcular: receita total, número de produtos vendidos e preço médio de venda por vendedor no banco de dados SQLite.